### NEUZZ Based Neural Network Fuzzing Script

In [1]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape data to fit model input requirements
x_train = x_train.reshape(-1, 28, 28, 1)  # Adding channel dimension
x_test = x_test.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a CNN model
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy}')

# Function to generate fuzzed inputs based on mutation strategies
def generate_fuzzing_inputs(base_image, num_samples=10):
    """Generate fuzzing inputs by applying various perturbation strategies to the base image."""
    fuzzed_inputs = []
    for _ in range(num_samples):
        # Randomly choose a mutation strategy
        mutation_type = np.random.choice(['noise', 'shift', 'scale'])
        
        if mutation_type == 'noise':
            # Add Gaussian noise
            perturbation = np.random.normal(0, 0.1, base_image.shape)
            fuzzed_image = np.clip(base_image + perturbation, 0, 1)
        
        elif mutation_type == 'shift':
            # Shift the image slightly
            shift_value = np.random.randint(-2, 3, size=(2,))  # Shift in height and width
            fuzzed_image = np.roll(base_image, shift_value[0], axis=0)  # Shift rows
            fuzzed_image = np.roll(fuzzed_image, shift_value[1], axis=1)  # Shift columns
        
        elif mutation_type == 'scale':
            # Scale the image slightly
            scale_factor = 1 + np.random.uniform(-0.1, 0.1)
            fuzzed_image = np.clip(base_image * scale_factor, 0, 1)
        
        fuzzed_inputs.append(fuzzed_image)

    return np.array(fuzzed_inputs)

# Choose a random base image from the test set
base_image = x_test[np.random.randint(0, len(x_test))]

# Generate fuzzing inputs
fuzzing_inputs = generate_fuzzing_inputs(base_image)

# Predict on fuzzed inputs and analyze results
predictions = []
for i, fuzzed_input in enumerate(fuzzing_inputs):
    fuzzed_input_reshaped = fuzzed_input.reshape(-1, 28, 28, 1)  # Reshape for model input
    prediction = model.predict(fuzzed_input_reshaped)
    predicted_label = np.argmax(prediction)
    predictions.append(predicted_label)
    print(f'Fuzzed Input {i+1} Predicted Label: {predicted_label}')

# Analyze prediction results
unique_predictions = set(predictions)
print(f'Unique Predicted Labels for Fuzzed Inputs: {unique_predictions}')

Epoch 1/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.1495 - accuracy: 0.9556 - val_loss: 0.0597 - val_accuracy: 0.9796
Epoch 2/5
1875/1875 [==============================] - 44s 23ms/step - loss: 0.0507 - accuracy: 0.9849 - val_loss: 0.0466 - val_accuracy: 0.9846
Epoch 3/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.0312 - accuracy: 0.9903 - val_loss: 0.0454 - val_accuracy: 0.9850
Epoch 4/5
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0212 - accuracy: 0.9928 - val_loss: 0.0401 - val_accuracy: 0.9874
Epoch 5/5
313/313 [==============================] - 2s 6ms/step - loss: 0.0458 - accuracy: 0.9862
Test accuracy: 0.9861999750137329
Fuzzed Input 1 Predicted Label: 9
Fuzzed Input 2 Predicted Label: 9
Fuzzed Input 3 Predicted Label: 9
Fuzzed Input 4 Predicted Label: 9
Fuzzed Input 5 Predicted Label: 0
Fuzzed Input 6 Predicted Label: 9
Fuzzed Input 7 Predicted Label: 9
Fuzzed Input 8 Predicted Label: 9
Fuzzed Input 9 

### Overall, this code effectively demonstrates a basic approach to neural network fuzzing by testing the model's responses to perturbed inputs. This can be useful for evaluating the model's resilience and understanding how it behaves under different conditions